# 1.- Definición General


## intención del colab

#### Incluye
*   Datos básicos del problema
*   Datos de insights del problema
*   Datos de Argentina (sin adivinar futuro)


#### NO incluye
*   Deltas, lags
*   relaciones estadísticas: medias, desvíos, min, max, ponderados y periódicos
*   Correlaciones entre series
*   Trabajos con NA / ceros





# 2.- Datos básicos del problema df_base_10

### Ambito general

In [ ]:
### me preparo en el entorno
# entorno
ColabNotebook = 'google.colab' in str(get_ipython())

if ColabNotebook:
    # monta G-drive en entorno COLAB
    from google.colab import drive
    drive.mount('/content/drive/')

    # carpeta donde se encuentran archivos .py auxiliares
    #FUENTES_DIR = '/content/drive/MyDrive/Colab Notebooks/FUENTES/'
    #DATOS_DIR = '/content/drive/MyDrive/Colab Notebooks/DATOS/'      # carpeta donde se encuentran los datasets
    FUENTES_DIR = '/content/drive/MyDrive/MCD/Labo3/FUENTES/' # Ajustar según usuario
    DATOS_DIR = '/content/drive/MyDrive/MCD_grupo17_la_desgracia/lab3/PruebasAK/DATOSL3/' # ajustar según usuario OJO crear un acceso directo para poder "montar" una unidad compartida
else:
    # configuración para notebook con instalación LOCAL
    # FUENTES_DIR = '../FUENTES'         # carpeta donde se encuentran archivos .py auxiliares
    DATOS_DIR   = './DATOS/' # carpeta donde se encuentran los datasets

# agrega ruta de busqueda donde tenemos archivos .py
import sys
sys.path.append(FUENTES_DIR)

Mounted at /content/drive/


In [ ]:
## Cargo lo necesario
# Cargo paquetes
#Librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#from numpy.polynomial import Polynomial
#from scipy.signal import detrend

import holidays

#Configuración de parametros de Pandas para mejor visualización
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.precision', 3)
pd.set_option('plotting.backend', 'matplotlib')

### Funciones útiles, gracias Diana Bermudez


In [ ]:
def eliminar_columnas_solo_ceros(df: pd.DataFrame) -> pd.DataFrame:
    """
    Elimina columnas de un DataFrame que contienen exclusivamente el valor cero.

    Args:
        df (pd.DataFrame): El DataFrame de entrada con múltiples columnas.

    Returns:
        pd.DataFrame: Un nuevo DataFrame con las columnas que no eran solo ceros.
                      Si no hay columnas restantes, devuelve un DataFrame vacío.
    """
    # Identificar las columnas que contienen solo ceros
    # Para cada columna, verificamos si la suma de sus valores absolutos es cero.
    # Esto maneja tanto ceros enteros como ceros flotantes.
    columnas_a_mantener = [col for col in df.columns if df[col].abs().sum() != 0]

    # Crear un nuevo DataFrame solo con las columnas a mantener
    df_resultante = df[columnas_a_mantener].copy()

    return df_resultante

In [ ]:
def reduce_mem_usage(df):
    """Itera por las columnas del DataFrame y modifica el tipo de datos para reducir uso de memoria."""
    start_mem = df.memory_usage().sum() / 1024**2
    print(f'Uso de memoria inicial del DataFrame: {start_mem:.2f} MB')

    for col in df.columns:
        col_type = df[col].dtype

        if pd.api.types.is_numeric_dtype(col_type):
            c_min = df[col].min()
            c_max = df[col].max()

            if pd.api.types.is_integer_dtype(col_type):
                if c_min >= np.iinfo(np.int8).min and c_max <= np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min >= np.iinfo(np.int16).min and c_max <= np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min >= np.iinfo(np.int32).min and c_max <= np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                else:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min >= np.finfo(np.float16).min and c_max <= np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min >= np.finfo(np.float32).min and c_max <= np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            # Sólo convertir a categoría si no lo es ya
            if not pd.api.types.is_categorical_dtype(df[col]):
                df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print(f'Uso de memoria final del DataFrame: {end_mem:.2f} MB')
    print(f'Memoria reducida en un {(100 * (start_mem - end_mem) / start_mem):.2f}%')
    return df

### Carga de archivos datos

In [ ]:
#cargar csv
# ventas al canal, va a ser el archivo padre
sellin = pd.read_csv(DATOS_DIR +'sell-in.txt', sep='\t')
# productos a validar, por ahora no se me ocurre como usarlos, pero los cargo por las dudas. Son los que se van a estar vivos
productos_validar = pd.read_csv(DATOS_DIR +'product_id_apredecir201912.txt', sep='\t')
# cargo la tabla de características
prod_cat = pd.read_csv(DATOS_DIR +'tb_productos.txt', sep='\t')
# cargo los stocks de los productos
stocks = pd.read_csv(DATOS_DIR +'tb_stocks.txt', sep='\t')

### proceso básico de sell-in => df_base_1

In [ ]:
print(sellin.shape)
sellin.head()

(2945818, 7)


,periodo,customer_id,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn
0,201701,10234,20524,0,2,0.053,0.053
1,201701,10032,20524,0,1,0.136,0.136
2,201701,10217,20524,0,1,0.030,0.030
3,201701,10125,20524,0,1,0.023,0.023
4,201701,10012,20524,0,11,1.545,1.545


In [ ]:
# lo primero que hago es eliminar cust_request_qty porque debe er la relación entre peso y cust request tn
# y si dejo cust_request_tn va a ser demasiando igual a tn, por lo que voy a dejar solamente dif_tn y dif_por como las diferencias absolutas y porcentuales sobre lo pedido
df_base_1 = sellin.drop(columns=['cust_request_qty'])
df_base_1['dif_tn'] = df_base_1['tn'] - df_base_1['cust_request_tn']
df_base_1['dif_por'] = df_base_1['dif_tn'] / df_base_1['cust_request_tn']
df_base_1 = df_base_1.drop(columns=['cust_request_tn'])
df_base_1.head()

,periodo,customer_id,product_id,plan_precios_cuidados,tn,dif_tn,dif_por
0,201701,10234,20524,0,0.053,0.0,0.0
1,201701,10032,20524,0,0.136,0.0,0.0
2,201701,10217,20524,0,0.030,0.0,0.0
3,201701,10125,20524,0,0.023,0.0,0.0
4,201701,10012,20524,0,1.545,0.0,0.0


In [ ]:
# plan precios cuidados es un binario

df_base_1['plan_precios_cuidados'] = df_base_1['plan_precios_cuidados'].astype(bool)
df_base_1.head()

,periodo,customer_id,product_id,plan_precios_cuidados,tn,dif_tn,dif_por
0,201701,10234,20524,False,0.053,0.0,0.0
1,201701,10032,20524,False,0.136,0.0,0.0
2,201701,10217,20524,False,0.030,0.0,0.0
3,201701,10125,20524,False,0.023,0.0,0.0
4,201701,10012,20524,False,1.545,0.0,0.0


In [ ]:
## de la columna periodo tengo que generar mes, año, trimestre, date(formato daytime), secuencia que es el mes consecutivo
## genero también una columna date_mes en formato período que me facilite la vida más adelante.
df_base_1['anio']=df_base_1['periodo'] //100
df_base_1['mes']=df_base_1['periodo'] % 100
df_base_1['secuencia']= (df_base_1['anio']-2017)*12+df_base_1['mes']
df_base_1['trimestre']=df_base_1['mes']//3
df_base_1['date'] = pd.to_datetime(df_base_1['periodo'], format='%Y%m')
df_base_1['date_mes'] = df_base_1['date'].dt.to_period('M')
df_base_1.head()


,periodo,customer_id,product_id,plan_precios_cuidados,tn,dif_tn,dif_por,anio,mes,secuencia,trimestre,date,date_mes
0,201701,10234,20524,False,0.053,0.0,0.0,2017,1,1,0,2017-01-01,2017-01
1,201701,10032,20524,False,0.136,0.0,0.0,2017,1,1,0,2017-01-01,2017-01
2,201701,10217,20524,False,0.030,0.0,0.0,2017,1,1,0,2017-01-01,2017-01
3,201701,10125,20524,False,0.023,0.0,0.0,2017,1,1,0,2017-01-01,2017-01
4,201701,10012,20524,False,1.545,0.0,0.0,2017,1,1,0,2017-01-01,2017-01


### datos básico categoría de productos y luego unión para df_base_2


In [ ]:
# busco y elimino registros duplicados
# Supongamos que tu columna se llama 'columna'
duplicados = prod_cat['product_id'].duplicated().any()
print("¿Hay duplicados?:", duplicados)
duplicados_valores = prod_cat['product_id'][prod_cat['product_id'].duplicated()]
print("Valores duplicados:")
print(duplicados_valores.unique())

¿Hay duplicados?: True
Valores duplicados:
[20010 20022 20021 20020 20278 20192 20037 20105 20100 20623 20230]


In [ ]:
#veo los duplicados a ver si amerita algún tratamiento o son simple error
duplicadas_filas = prod_cat[prod_cat['product_id'].duplicated(keep=False)]
print(duplicadas_filas)

      cat1            cat2            cat3     brand  sku_size  product_id
36      HC     ROPA LAVADO           Polvo   LIMPIEX       400       20010
37      HC     ROPA LAVADO           Polvo   LIMPIEX       400       20010
38      HC     ROPA LAVADO           Polvo   LIMPIEX       800       20022
39      HC     ROPA LAVADO           Polvo   LIMPIEX       800       20022
41      HC     ROPA LAVADO           Polvo   LIMPIEX       400       20021
42      HC     ROPA LAVADO           Polvo   LIMPIEX       400       20021
43      HC     ROPA LAVADO           Polvo   LIMPIEX       800       20020
44      HC     ROPA LAVADO           Polvo   LIMPIEX       800       20020
541  FOODS  SOPAS Y CALDOS      Caldo Cubo     MAGGI         2       20278
542  FOODS  SOPAS Y CALDOS      Caldo Cubo     MAGGI         2       20278
550  FOODS  SOPAS Y CALDOS      Caldo Cubo     MAGGI         2       20192
551  FOODS  SOPAS Y CALDOS      Caldo Cubo     MAGGI         2       20192
552  FOODS  SOPAS Y CALDO

In [ ]:
prod_cat.isnull().sum()

,0
cat1,0
cat2,0
cat3,0
brand,0
sku_size,0
product_id,0


In [ ]:
#elimino los duplicados
# el peor caso es un shampoo por un acondicionador... no cambia mucho
prod_cat_unic = prod_cat.drop_duplicates(subset=['product_id'])
duplicados = prod_cat_unic['product_id'].duplicated().any()
print("¿Hay duplicados?:", duplicados)
duplicados_valores = prod_cat_unic['product_id'][prod_cat_unic['product_id'].duplicated()]
print("Valores duplicados:")
print(duplicados_valores.unique())

¿Hay duplicados?: False
Valores duplicados:
[]


In [ ]:
#voy metiendo todo en una sola matriz,
# que me llevo
columnas_a_agregar = ['cat1', 'cat2', 'cat3','brand','sku_size', 'product_id']
# resultado
df_base_2 = df_base_1.merge(prod_cat_unic[columnas_a_agregar], on='product_id', how='left')

df_base_2.head()

,periodo,customer_id,product_id,plan_precios_cuidados,tn,dif_tn,dif_por,anio,mes,secuencia,trimestre,date,date_mes,cat1,cat2,cat3,brand,sku_size
0,201701,10234,20524,False,0.053,0.0,0.0,2017,1,1,0,2017-01-01,2017-01,HC,VAJILLA,Cristalino,Importado,500.0
1,201701,10032,20524,False,0.136,0.0,0.0,2017,1,1,0,2017-01-01,2017-01,HC,VAJILLA,Cristalino,Importado,500.0
2,201701,10217,20524,False,0.030,0.0,0.0,2017,1,1,0,2017-01-01,2017-01,HC,VAJILLA,Cristalino,Importado,500.0
3,201701,10125,20524,False,0.023,0.0,0.0,2017,1,1,0,2017-01-01,2017-01,HC,VAJILLA,Cristalino,Importado,500.0
4,201701,10012,20524,False,1.545,0.0,0.0,2017,1,1,0,2017-01-01,2017-01,HC,VAJILLA,Cristalino,Importado,500.0


In [ ]:
df_base_2.shape

(2945818, 18)

In [ ]:
df_base_2.isnull().sum()

,0
periodo,0
customer_id,0
product_id,0
plan_precios_cuidados,0
tn,0
dif_tn,0
dif_por,0
anio,0
mes,0
secuencia,0


In [ ]:
# Agrupa y cuenta productos con categoría nula, y ordena de mayor a menor
conteo = (
    df_base_2[df_base_2['cat1'].isnull()]
    .groupby('product_id')
    .size()
    .sort_values(ascending=False)
)

print(conteo)

product_id
20848    2628
20808    1409
21125    1323
21217     318
21165     294
21098     249
21178     247
21241     185
21225     177
21240     141
21253     111
21272      67
21223      49
21238      43
21199      38
21066      27
21273      23
21268      12
21228      12
21277      11
20918      11
21283      11
21275       8
21249       8
21230       8
21281       6
21274       6
21285       4
21286       2
21290       2
21279       2
21278       2
21284       2
21169       1
21270       1
21288       1
21289       1
21291       1
21292       1
21293       1
21295       1
21296       1
21297       1
21298       1
21299       1
dtype: int64


In [ ]:
# tengo el problema que hay 7448 instancias donde no puedo asociar producto con características
# posiblement los productos que fueron mal colocados
# los voy a reemplazar por UNKN por desconocido
columnas_raras =['cat1', 'cat2', 'cat3','brand','sku_size']
df_base_2[columnas_raras] = df_base_2[columnas_raras].fillna('UNKN')
df_base_2.isnull().sum()
#

,0
periodo,0
customer_id,0
product_id,0
plan_precios_cuidados,0
tn,0
dif_tn,0
dif_por,0
anio,0
mes,0
secuencia,0


# 3.- Datos de insights del problema

### en clase se habló de ver hermanos mayores y menores, mismo producto distinto size, no queda muy claro como hacerlo a partir de la codificación, por ahora no lo hago


### la estructura de categorías, el árbol no me resulta claro, por lo que voy a crear nuevas categorías que lo reflejen df_base_3


In [ ]:
# cuando yo voy partiendo en categorías la idea es ir quedandome con subconjuntos lo más parecidos posibles
# y nada me garantiza la unicidad o correspondencia de las catagorías que aquí aparecen más abajo
# por ejemplo puedo tener un jabón de 500 ml  para bebé y uno de lavar la ropa de 500ml, no me hace sentido agruparlos solamente por su size
# entonces voy a generar Categorías a partir de la 2 que sean una concatenación de los valores de más arriba en la jerarquia
df_base_3=df_base_2.copy()
df_base_3['cat1_cat2'] = df_base_3['cat1'].fillna('') + '_' + df_base_3['cat2'].fillna('')
df_base_3['cat1_cat2_cat3'] = df_base_3['cat1_cat2'].fillna('') + '_' + df_base_3['cat3'].fillna('')
df_base_3['cat1_cat2_cat3_brand'] = df_base_3['cat1_cat2_cat3'].fillna('') + '_' + df_base_3['brand'].fillna('')
df_base_3['cat1_cat2_cat3_brand_size'] = df_base_3['cat1_cat2_cat3_brand'].fillna('') + '_' + df_base_3['sku_size'].astype(str)
df_base_3.head()

,periodo,customer_id,product_id,plan_precios_cuidados,tn,dif_tn,dif_por,anio,mes,secuencia,trimestre,date,date_mes,cat1,cat2,cat3,brand,sku_size,cat1_cat2,cat1_cat2_cat3,cat1_cat2_cat3_brand,cat1_cat2_cat3_brand_size
0,201701,10234,20524,False,0.053,0.0,0.0,2017,1,1,0,2017-01-01,2017-01,HC,VAJILLA,Cristalino,Importado,500.0,HC_VAJILLA,HC_VAJILLA_Cristalino,HC_VAJILLA_Cristalino_Importado,HC_VAJILLA_Cristalino_Importado_500.0
1,201701,10032,20524,False,0.136,0.0,0.0,2017,1,1,0,2017-01-01,2017-01,HC,VAJILLA,Cristalino,Importado,500.0,HC_VAJILLA,HC_VAJILLA_Cristalino,HC_VAJILLA_Cristalino_Importado,HC_VAJILLA_Cristalino_Importado_500.0
2,201701,10217,20524,False,0.030,0.0,0.0,2017,1,1,0,2017-01-01,2017-01,HC,VAJILLA,Cristalino,Importado,500.0,HC_VAJILLA,HC_VAJILLA_Cristalino,HC_VAJILLA_Cristalino_Importado,HC_VAJILLA_Cristalino_Importado_500.0
3,201701,10125,20524,False,0.023,0.0,0.0,2017,1,1,0,2017-01-01,2017-01,HC,VAJILLA,Cristalino,Importado,500.0,HC_VAJILLA,HC_VAJILLA_Cristalino,HC_VAJILLA_Cristalino_Importado,HC_VAJILLA_Cristalino_Importado_500.0
4,201701,10012,20524,False,1.545,0.0,0.0,2017,1,1,0,2017-01-01,2017-01,HC,VAJILLA,Cristalino,Importado,500.0,HC_VAJILLA,HC_VAJILLA_Cristalino,HC_VAJILLA_Cristalino_Importado,HC_VAJILLA_Cristalino_Importado_500.0


In [ ]:
# saco las columnas que ahora son redundantes
df_base_3 = df_base_3.drop(columns=[ 'cat2', 'cat3','brand','sku_size'])
df_base_3.head()

,periodo,customer_id,product_id,plan_precios_cuidados,tn,dif_tn,dif_por,anio,mes,secuencia,trimestre,date,date_mes,cat1,cat1_cat2,cat1_cat2_cat3,cat1_cat2_cat3_brand,cat1_cat2_cat3_brand_size
0,201701,10234,20524,False,0.053,0.0,0.0,2017,1,1,0,2017-01-01,2017-01,HC,HC_VAJILLA,HC_VAJILLA_Cristalino,HC_VAJILLA_Cristalino_Importado,HC_VAJILLA_Cristalino_Importado_500.0
1,201701,10032,20524,False,0.136,0.0,0.0,2017,1,1,0,2017-01-01,2017-01,HC,HC_VAJILLA,HC_VAJILLA_Cristalino,HC_VAJILLA_Cristalino_Importado,HC_VAJILLA_Cristalino_Importado_500.0
2,201701,10217,20524,False,0.030,0.0,0.0,2017,1,1,0,2017-01-01,2017-01,HC,HC_VAJILLA,HC_VAJILLA_Cristalino,HC_VAJILLA_Cristalino_Importado,HC_VAJILLA_Cristalino_Importado_500.0
3,201701,10125,20524,False,0.023,0.0,0.0,2017,1,1,0,2017-01-01,2017-01,HC,HC_VAJILLA,HC_VAJILLA_Cristalino,HC_VAJILLA_Cristalino_Importado,HC_VAJILLA_Cristalino_Importado_500.0
4,201701,10012,20524,False,1.545,0.0,0.0,2017,1,1,0,2017-01-01,2017-01,HC,HC_VAJILLA,HC_VAJILLA_Cristalino,HC_VAJILLA_Cristalino_Importado,HC_VAJILLA_Cristalino_Importado_500.0


In [ ]:
# agrego stock a fin de mes por producto, no me interesa en esta instancia generar los deltas , medias etc)
print(stocks.tail())

       periodo  product_id  stock_final
13686   201912       20453        1.437
13687   201912       21026        7.268
13688   201912       21054        0.508
13689   201912       20981        2.185
13690   201912       20853        1.824


In [ ]:
# 2. Añadir el stock. La clave de unión es el mes Y el producto.
df_base_3 = pd.merge(df_base_3, stocks, on=['periodo', 'product_id'], how='left')
df_base_3.tail()

,periodo,customer_id,product_id,plan_precios_cuidados,tn,dif_tn,dif_por,anio,mes,secuencia,trimestre,date,date_mes,cat1,cat1_cat2,cat1_cat2_cat3,cat1_cat2_cat3_brand,cat1_cat2_cat3_brand_size,stock_final
2945813,201912,10105,20853,False,0.022,0.0,0.0,2019,12,36,4,2019-12-01,2019-12,PC,PC_CABELLO,PC_CABELLO_Shampoo Bebe,PC_CABELLO_Shampoo Bebe_NIVEA,PC_CABELLO_Shampoo Bebe_NIVEA_200.0,1.824
2945814,201912,10092,20853,False,0.007,0.0,0.0,2019,12,36,4,2019-12-01,2019-12,PC,PC_CABELLO,PC_CABELLO_Shampoo Bebe,PC_CABELLO_Shampoo Bebe_NIVEA,PC_CABELLO_Shampoo Bebe_NIVEA_200.0,1.824
2945815,201912,10006,20853,False,0.029,0.0,0.0,2019,12,36,4,2019-12-01,2019-12,PC,PC_CABELLO,PC_CABELLO_Shampoo Bebe,PC_CABELLO_Shampoo Bebe_NIVEA,PC_CABELLO_Shampoo Bebe_NIVEA_200.0,1.824
2945816,201912,10018,20853,False,0.016,0.0,0.0,2019,12,36,4,2019-12-01,2019-12,PC,PC_CABELLO,PC_CABELLO_Shampoo Bebe,PC_CABELLO_Shampoo Bebe_NIVEA,PC_CABELLO_Shampoo Bebe_NIVEA_200.0,1.824
2945817,201912,10020,20853,False,0.016,0.0,0.0,2019,12,36,4,2019-12-01,2019-12,PC,PC_CABELLO,PC_CABELLO_Shampoo Bebe,PC_CABELLO_Shampoo Bebe_NIVEA,PC_CABELLO_Shampoo Bebe_NIVEA_200.0,1.824


### ahora creo los productos

### ahora tengo que ver los períodos de vida y "muerte" para cada producto df_base_4
#### voy a generar algunos archivos que van a quedar guardados como auxiliares

In [ ]:
# busco y guardo la fecha de nacimiento de producto, la de cliente y la muerte de producto. Los clientes los considero siempre vivos
# Agrupamos por el identificador del producto
# Luego, sobre la columna 'periodo', aplicamos las agregaciones min y max
# las fechas las dejo en período porque me interesa solamente sumar y restar luego
fechas_venta_producto = df_base_3.groupby('product_id')['date'].agg(
    primera_venta='min',  # Renombramos la columna 'min' a 'primera_venta'
    ultima_venta='max'    # Renombramos la columna 'max' a 'ultima_venta'
).reset_index() # Convertimos el índice (id_producto) de nuevo a una columna
fechas_venta_producto['primera_venta'] = fechas_venta_producto['primera_venta'].dt.to_period('M')
fechas_venta_producto['ultima_venta'] = fechas_venta_producto['ultima_venta'].dt.to_period('M')
print(fechas_venta_producto.head())
fechas_venta_producto.to_csv(DATOS_DIR +'fechas_venta_producto.csv', index=False)

   product_id primera_venta ultima_venta
0       20001       2017-01      2019-12
1       20002       2017-01      2019-12
2       20003       2017-01      2019-12
3       20004       2017-01      2019-12
4       20005       2017-01      2019-12


In [ ]:
# busco y guardo la fecha de nacimiento de producto, la de cliente y la muerte de producto. Los clientes los considero siempre vivos
# Agrupamos por el identificador del producto
# Luego, sobre la columna 'periodo', aplicamos las agregaciones min y max
# las fechas las dejo en período porque me interesa solamente sumar y restar luego
fechas_alta_cliente = df_base_3.groupby('customer_id')['date'].agg(
    alta_cliente='min',  # Renombramos la columna 'min' a 'alta_cliente'
    ultima_compra='max'    # Renombramos la columna 'max' a 'ultima_compra'
).reset_index() # Convertimos el índice (id_producto) de nuevo a una columna
fechas_alta_cliente['alta_cliente'] = fechas_alta_cliente['alta_cliente'].dt.to_period('M')
fechas_alta_cliente['ultima_compra'] = fechas_alta_cliente['ultima_compra'].dt.to_period('M')
fechas_alta_cliente.to_csv(DATOS_DIR +'fechas_altas_cliente.csv', index=False)
print(fechas_alta_cliente.head())

   customer_id alta_cliente ultima_compra
0        10001      2017-01       2019-12
1        10002      2017-01       2019-12
2        10003      2017-01       2019-12
3        10004      2017-01       2019-12
4        10005      2017-01       2019-12


In [ ]:
# voy a crear df_base_4, le agrego todos los datos a derecha, hago las cuentas y luego suprimo las columnas
df_base_4 = df_base_3.copy()

In [ ]:
#merge con product_id de fechas_venta_producto
df_base_4 = pd.merge(df_base_4, fechas_venta_producto, on='product_id', how='left')
#merge con client_id de fechas_alta_cliente
df_base_4 = pd.merge(df_base_4, fechas_alta_cliente, on='customer_id', how='left')
df_base_4.tail()

,periodo,customer_id,product_id,plan_precios_cuidados,tn,dif_tn,dif_por,anio,mes,secuencia,trimestre,date,date_mes,cat1,cat1_cat2,cat1_cat2_cat3,cat1_cat2_cat3_brand,cat1_cat2_cat3_brand_size,stock_final,primera_venta,ultima_venta,alta_cliente,ultima_compra
2945813,201912,10105,20853,False,0.022,0.0,0.0,2019,12,36,4,2019-12-01,2019-12,PC,PC_CABELLO,PC_CABELLO_Shampoo Bebe,PC_CABELLO_Shampoo Bebe_NIVEA,PC_CABELLO_Shampoo Bebe_NIVEA_200.0,1.824,2018-08,2019-12,2017-01,2019-12
2945814,201912,10092,20853,False,0.007,0.0,0.0,2019,12,36,4,2019-12-01,2019-12,PC,PC_CABELLO,PC_CABELLO_Shampoo Bebe,PC_CABELLO_Shampoo Bebe_NIVEA,PC_CABELLO_Shampoo Bebe_NIVEA_200.0,1.824,2018-08,2019-12,2017-01,2019-12
2945815,201912,10006,20853,False,0.029,0.0,0.0,2019,12,36,4,2019-12-01,2019-12,PC,PC_CABELLO,PC_CABELLO_Shampoo Bebe,PC_CABELLO_Shampoo Bebe_NIVEA,PC_CABELLO_Shampoo Bebe_NIVEA_200.0,1.824,2018-08,2019-12,2017-01,2019-12
2945816,201912,10018,20853,False,0.016,0.0,0.0,2019,12,36,4,2019-12-01,2019-12,PC,PC_CABELLO,PC_CABELLO_Shampoo Bebe,PC_CABELLO_Shampoo Bebe_NIVEA,PC_CABELLO_Shampoo Bebe_NIVEA_200.0,1.824,2018-08,2019-12,2017-01,2019-12
2945817,201912,10020,20853,False,0.016,0.0,0.0,2019,12,36,4,2019-12-01,2019-12,PC,PC_CABELLO,PC_CABELLO_Shampoo Bebe,PC_CABELLO_Shampoo Bebe_NIVEA,PC_CABELLO_Shampoo Bebe_NIVEA_200.0,1.824,2018-08,2019-12,2017-01,2019-12


In [ ]:
df_base_4.isnull().sum()

,0
periodo,0
customer_id,0
product_id,0
plan_precios_cuidados,0
tn,0
dif_tn,0
dif_por,0
anio,0
mes,0
secuencia,0


In [ ]:
# se que productos validar estará vivo en 2020-02-01
productos_validar['vive'] = pd.to_datetime('2020-02-01')
productos_validar['vive'] = productos_validar['vive'].dt.to_period('M')
print(productos_validar.head())

   product_id     vive
0       20001  2020-02
1       20002  2020-02
2       20003  2020-02
3       20004  2020-02
4       20005  2020-02


In [ ]:
# merge con product_id de df_base_4 con productos_validar
df_base_4 = pd.merge(df_base_4, productos_validar, on='product_id', how='left')
df_base_4.tail()


,periodo,customer_id,product_id,plan_precios_cuidados,tn,dif_tn,dif_por,anio,mes,secuencia,trimestre,date,date_mes,cat1,cat1_cat2,cat1_cat2_cat3,cat1_cat2_cat3_brand,cat1_cat2_cat3_brand_size,stock_final,primera_venta,ultima_venta,alta_cliente,ultima_compra,vive
2945813,201912,10105,20853,False,0.022,0.0,0.0,2019,12,36,4,2019-12-01,2019-12,PC,PC_CABELLO,PC_CABELLO_Shampoo Bebe,PC_CABELLO_Shampoo Bebe_NIVEA,PC_CABELLO_Shampoo Bebe_NIVEA_200.0,1.824,2018-08,2019-12,2017-01,2019-12,2020-02
2945814,201912,10092,20853,False,0.007,0.0,0.0,2019,12,36,4,2019-12-01,2019-12,PC,PC_CABELLO,PC_CABELLO_Shampoo Bebe,PC_CABELLO_Shampoo Bebe_NIVEA,PC_CABELLO_Shampoo Bebe_NIVEA_200.0,1.824,2018-08,2019-12,2017-01,2019-12,2020-02
2945815,201912,10006,20853,False,0.029,0.0,0.0,2019,12,36,4,2019-12-01,2019-12,PC,PC_CABELLO,PC_CABELLO_Shampoo Bebe,PC_CABELLO_Shampoo Bebe_NIVEA,PC_CABELLO_Shampoo Bebe_NIVEA_200.0,1.824,2018-08,2019-12,2017-01,2019-12,2020-02
2945816,201912,10018,20853,False,0.016,0.0,0.0,2019,12,36,4,2019-12-01,2019-12,PC,PC_CABELLO,PC_CABELLO_Shampoo Bebe,PC_CABELLO_Shampoo Bebe_NIVEA,PC_CABELLO_Shampoo Bebe_NIVEA_200.0,1.824,2018-08,2019-12,2017-01,2019-12,2020-02
2945817,201912,10020,20853,False,0.016,0.0,0.0,2019,12,36,4,2019-12-01,2019-12,PC,PC_CABELLO,PC_CABELLO_Shampoo Bebe,PC_CABELLO_Shampoo Bebe_NIVEA,PC_CABELLO_Shampoo Bebe_NIVEA_200.0,1.824,2018-08,2019-12,2017-01,2019-12,2020-02


In [ ]:
print(df_base_4.head())

   periodo  customer_id  product_id  plan_precios_cuidados     tn  dif_tn  \
0   201701        10234       20524                  False  0.053     0.0   
1   201701        10032       20524                  False  0.136     0.0   
2   201701        10217       20524                  False  0.030     0.0   
3   201701        10125       20524                  False  0.023     0.0   
4   201701        10012       20524                  False  1.545     0.0   

   dif_por  anio  mes  secuencia  trimestre       date date_mes cat1  \
0      0.0  2017    1          1          0 2017-01-01  2017-01   HC   
1      0.0  2017    1          1          0 2017-01-01  2017-01   HC   
2      0.0  2017    1          1          0 2017-01-01  2017-01   HC   
3      0.0  2017    1          1          0 2017-01-01  2017-01   HC   
4      0.0  2017    1          1          0 2017-01-01  2017-01   HC   

    cat1_cat2         cat1_cat2_cat3             cat1_cat2_cat3_brand  \
0  HC_VAJILLA  HC_VAJILLA_Crist

In [ ]:
df_base_5=df_base_4.copy()

In [ ]:
# months of product life,
# OJO, por definición si primera venta  o primera compra es en 2017 01, entonces considero que comenzaron tres meses antes, salen con valor 4
# para el resto es una diferencia simple, pero no necesito sumar 4
fecha_base = pd.Period('2017-01', freq='M')
df_base_5['meses_vida_producto'] = (df_base_5['date_mes'].astype(int) - df_base_5['primera_venta'].astype(int))+1
df_base_5['meses_vida_cliente'] = (df_base_5['date_mes'].astype(int) - df_base_5['alta_cliente'].astype(int))+1
# Si el producto empezó en el primer periodo (2017-01), suma 3 para que inicie en 4
df_base_5.loc[df_base_5['primera_venta'] == fecha_base, 'meses_vida_producto'] += 3
# Si el cliente empezó en el primer periodo (2017-01), suma 3 para que inicie en 4
df_base_5.loc[df_base_5['alta_cliente'] == fecha_base, 'meses_vida_cliente'] += 3
df_base_5.head()

,periodo,customer_id,product_id,plan_precios_cuidados,tn,dif_tn,dif_por,anio,mes,secuencia,trimestre,date,date_mes,cat1,cat1_cat2,cat1_cat2_cat3,cat1_cat2_cat3_brand,cat1_cat2_cat3_brand_size,stock_final,primera_venta,ultima_venta,alta_cliente,ultima_compra,vive,meses_vida_producto,meses_vida_cliente
0,201701,10234,20524,False,0.053,0.0,0.0,2017,1,1,0,2017-01-01,2017-01,HC,HC_VAJILLA,HC_VAJILLA_Cristalino,HC_VAJILLA_Cristalino_Importado,HC_VAJILLA_Cristalino_Importado_500.0,NaN,2017-01,2019-12,2017-01,2019-12,2020-02,4,4
1,201701,10032,20524,False,0.136,0.0,0.0,2017,1,1,0,2017-01-01,2017-01,HC,HC_VAJILLA,HC_VAJILLA_Cristalino,HC_VAJILLA_Cristalino_Importado,HC_VAJILLA_Cristalino_Importado_500.0,NaN,2017-01,2019-12,2017-01,2019-12,2020-02,4,4
2,201701,10217,20524,False,0.030,0.0,0.0,2017,1,1,0,2017-01-01,2017-01,HC,HC_VAJILLA,HC_VAJILLA_Cristalino,HC_VAJILLA_Cristalino_Importado,HC_VAJILLA_Cristalino_Importado_500.0,NaN,2017-01,2019-12,2017-01,2019-12,2020-02,4,4
3,201701,10125,20524,False,0.023,0.0,0.0,2017,1,1,0,2017-01-01,2017-01,HC,HC_VAJILLA,HC_VAJILLA_Cristalino,HC_VAJILLA_Cristalino_Importado,HC_VAJILLA_Cristalino_Importado_500.0,NaN,2017-01,2019-12,2017-01,2019-11,2020-02,4,4
4,201701,10012,20524,False,1.545,0.0,0.0,2017,1,1,0,2017-01-01,2017-01,HC,HC_VAJILLA,HC_VAJILLA_Cristalino,HC_VAJILLA_Cristalino_Importado,HC_VAJILLA_Cristalino_Importado_500.0,NaN,2017-01,2019-12,2017-01,2019-12,2020-02,4,4


In [ ]:
df_base_5.tail()

,periodo,customer_id,product_id,plan_precios_cuidados,tn,dif_tn,dif_por,anio,mes,secuencia,trimestre,date,date_mes,cat1,cat1_cat2,cat1_cat2_cat3,cat1_cat2_cat3_brand,cat1_cat2_cat3_brand_size,stock_final,primera_venta,ultima_venta,alta_cliente,ultima_compra,vive,meses_vida_producto,meses_vida_cliente
2945813,201912,10105,20853,False,0.022,0.0,0.0,2019,12,36,4,2019-12-01,2019-12,PC,PC_CABELLO,PC_CABELLO_Shampoo Bebe,PC_CABELLO_Shampoo Bebe_NIVEA,PC_CABELLO_Shampoo Bebe_NIVEA_200.0,1.824,2018-08,2019-12,2017-01,2019-12,2020-02,17,39
2945814,201912,10092,20853,False,0.007,0.0,0.0,2019,12,36,4,2019-12-01,2019-12,PC,PC_CABELLO,PC_CABELLO_Shampoo Bebe,PC_CABELLO_Shampoo Bebe_NIVEA,PC_CABELLO_Shampoo Bebe_NIVEA_200.0,1.824,2018-08,2019-12,2017-01,2019-12,2020-02,17,39
2945815,201912,10006,20853,False,0.029,0.0,0.0,2019,12,36,4,2019-12-01,2019-12,PC,PC_CABELLO,PC_CABELLO_Shampoo Bebe,PC_CABELLO_Shampoo Bebe_NIVEA,PC_CABELLO_Shampoo Bebe_NIVEA_200.0,1.824,2018-08,2019-12,2017-01,2019-12,2020-02,17,39
2945816,201912,10018,20853,False,0.016,0.0,0.0,2019,12,36,4,2019-12-01,2019-12,PC,PC_CABELLO,PC_CABELLO_Shampoo Bebe,PC_CABELLO_Shampoo Bebe_NIVEA,PC_CABELLO_Shampoo Bebe_NIVEA_200.0,1.824,2018-08,2019-12,2017-01,2019-12,2020-02,17,39
2945817,201912,10020,20853,False,0.016,0.0,0.0,2019,12,36,4,2019-12-01,2019-12,PC,PC_CABELLO,PC_CABELLO_Shampoo Bebe,PC_CABELLO_Shampoo Bebe_NIVEA,PC_CABELLO_Shampoo Bebe_NIVEA_200.0,1.824,2018-08,2019-12,2017-01,2019-12,2020-02,17,39


In [ ]:
df_base_5.shape

(2945818, 26)

In [ ]:
# Por defecto, el tiempo de vida de un producto es NaN, sigue vendiéndose
# si deja de venderse por X meses, voy a tomar que se vendió por última vez el mes que faltó de mercado
# Vamos a explorar primero ventas por mes por producto en cero a ver que encontramos


# Suponemos que tienes el DataFrame original 'df' y las columnas product_id, mes y (opcional) ventas


todos_los_productos = df_base_5['product_id'].unique()
todos_los_meses = df_base_5['date_mes'].unique()

In [ ]:

idx = pd.MultiIndex.from_product([todos_los_productos, todos_los_meses], names=['product_id', 'date_mes'])
df_full = pd.DataFrame(index=idx).reset_index()

In [ ]:



if 'tn' in df_base_5.columns:
    ventas_por_prod_mes = df_base_5.groupby(['product_id', 'date_mes'])['tn'].sum().reset_index()
else:
    ventas_por_prod_mes = df_base_5.groupby(['product_id', 'date_mes']).size().reset_index(name='tn')

resumen = pd.merge(df_full, ventas_por_prod_mes, on=['product_id', 'date_mes'], how='left')
resumen['tn'] = resumen['tn'].fillna(0)

# Filtra sólo los meses con ventas cero
productos_cero_ventas = resumen[resumen['tn'] == 0]

# Ahora agrupa por producto y lista los meses en una columna
resumen_agrupado = productos_cero_ventas.groupby('product_id')['date_mes'].apply(list).reset_index()

print(resumen_agrupado)

     product_id  \
0         20032   
1         20034   
2         20036   
3         20040   
4         20049   
5         20060   
6         20064   
7         20083   
8         20085   
9         20089   
10        20098   
11        20104   
12        20110   
13        20112   
14        20126   
15        20127   
16        20128   
17        20130   
18        20131   
19        20135   
20        20141   
21        20143   
22        20147   
23        20149   
24        20150   
25        20154   
26        20156   
27        20159   
28        20164   
29        20170   
30        20172   
31        20174   
32        20186   
33        20191   
34        20192   
35        20195   
36        20199   
37        20202   
38        20203   
39        20210   
40        20213   
41        20214   
42        20217   
43        20218   
44        20221   
45        20223   
46        20229   
47        20236   
48        20237   
49        20243   
50        20245   
51        20

In [ ]:
# en una inspección rápida, parece que cuando muere, muere, no hay períodos de cero venta y luego inicio d eventa de nuevo
df_base_5.tail()

,periodo,customer_id,product_id,plan_precios_cuidados,tn,dif_tn,dif_por,anio,mes,secuencia,trimestre,date,date_mes,cat1,cat1_cat2,cat1_cat2_cat3,cat1_cat2_cat3_brand,cat1_cat2_cat3_brand_size,stock_final,primera_venta,ultima_venta,alta_cliente,ultima_compra,vive,meses_vida_producto,meses_vida_cliente
2945813,201912,10105,20853,False,0.022,0.0,0.0,2019,12,36,4,2019-12-01,2019-12,PC,PC_CABELLO,PC_CABELLO_Shampoo Bebe,PC_CABELLO_Shampoo Bebe_NIVEA,PC_CABELLO_Shampoo Bebe_NIVEA_200.0,1.824,2018-08,2019-12,2017-01,2019-12,2020-02,17,39
2945814,201912,10092,20853,False,0.007,0.0,0.0,2019,12,36,4,2019-12-01,2019-12,PC,PC_CABELLO,PC_CABELLO_Shampoo Bebe,PC_CABELLO_Shampoo Bebe_NIVEA,PC_CABELLO_Shampoo Bebe_NIVEA_200.0,1.824,2018-08,2019-12,2017-01,2019-12,2020-02,17,39
2945815,201912,10006,20853,False,0.029,0.0,0.0,2019,12,36,4,2019-12-01,2019-12,PC,PC_CABELLO,PC_CABELLO_Shampoo Bebe,PC_CABELLO_Shampoo Bebe_NIVEA,PC_CABELLO_Shampoo Bebe_NIVEA_200.0,1.824,2018-08,2019-12,2017-01,2019-12,2020-02,17,39
2945816,201912,10018,20853,False,0.016,0.0,0.0,2019,12,36,4,2019-12-01,2019-12,PC,PC_CABELLO,PC_CABELLO_Shampoo Bebe,PC_CABELLO_Shampoo Bebe_NIVEA,PC_CABELLO_Shampoo Bebe_NIVEA_200.0,1.824,2018-08,2019-12,2017-01,2019-12,2020-02,17,39
2945817,201912,10020,20853,False,0.016,0.0,0.0,2019,12,36,4,2019-12-01,2019-12,PC,PC_CABELLO,PC_CABELLO_Shampoo Bebe,PC_CABELLO_Shampoo Bebe_NIVEA,PC_CABELLO_Shampoo Bebe_NIVEA_200.0,1.824,2018-08,2019-12,2017-01,2019-12,2020-02,17,39


In [ ]:
# meses por vivir, NA para lo que salga más allá de la predicción. Si hay productos que no aparecen en la predicción, viven hasta el mes anterior
# condiciones de valor tiempo de vida de cliente
# Si vive tiene valor 2020-02 => tiempo de vida = NaN
# si vive no tiene el valor 2020-02 y ultima venta 2019-12  entonces 2020-01
# si no ultima vcompra menos date




fecha_dato = pd.Period('2020-02', freq='M')  # lo que espero encontrar en vive
fecha_dato_2 = pd.Period('2020-01', freq='M') # lo que no vive y vivía en 2019 12
fecha_dato_3 = pd.Period('2019-12', freq='M')
diferencia_larga = fecha_dato_2.ordinal - df_base_5['date_mes'].astype(int)
diferencia_corta = df_base_5['ultima_venta'].astype(int)- df_base_5['date_mes'].astype(int)

conds = [
    (df_base_5['vive'] == fecha_dato),
    (df_base_5['vive'] != fecha_dato) & (df_base_5['ultima_venta'] == fecha_dato_3)
]
choices = [
    np.nan,
    diferencia_larga
]
default = diferencia_corta
df_base_5['meses_vida_prod'] = np.select(conds, choices, default)
df_base_5.head()

,periodo,customer_id,product_id,plan_precios_cuidados,tn,dif_tn,dif_por,anio,mes,secuencia,trimestre,date,date_mes,cat1,cat1_cat2,cat1_cat2_cat3,cat1_cat2_cat3_brand,cat1_cat2_cat3_brand_size,stock_final,primera_venta,ultima_venta,alta_cliente,ultima_compra,vive,meses_vida_producto,meses_vida_cliente,meses_vida_prod
0,201701,10234,20524,False,0.053,0.0,0.0,2017,1,1,0,2017-01-01,2017-01,HC,HC_VAJILLA,HC_VAJILLA_Cristalino,HC_VAJILLA_Cristalino_Importado,HC_VAJILLA_Cristalino_Importado_500.0,NaN,2017-01,2019-12,2017-01,2019-12,2020-02,4,4,NaN
1,201701,10032,20524,False,0.136,0.0,0.0,2017,1,1,0,2017-01-01,2017-01,HC,HC_VAJILLA,HC_VAJILLA_Cristalino,HC_VAJILLA_Cristalino_Importado,HC_VAJILLA_Cristalino_Importado_500.0,NaN,2017-01,2019-12,2017-01,2019-12,2020-02,4,4,NaN
2,201701,10217,20524,False,0.030,0.0,0.0,2017,1,1,0,2017-01-01,2017-01,HC,HC_VAJILLA,HC_VAJILLA_Cristalino,HC_VAJILLA_Cristalino_Importado,HC_VAJILLA_Cristalino_Importado_500.0,NaN,2017-01,2019-12,2017-01,2019-12,2020-02,4,4,NaN
3,201701,10125,20524,False,0.023,0.0,0.0,2017,1,1,0,2017-01-01,2017-01,HC,HC_VAJILLA,HC_VAJILLA_Cristalino,HC_VAJILLA_Cristalino_Importado,HC_VAJILLA_Cristalino_Importado_500.0,NaN,2017-01,2019-12,2017-01,2019-11,2020-02,4,4,NaN
4,201701,10012,20524,False,1.545,0.0,0.0,2017,1,1,0,2017-01-01,2017-01,HC,HC_VAJILLA,HC_VAJILLA_Cristalino,HC_VAJILLA_Cristalino_Importado,HC_VAJILLA_Cristalino_Importado_500.0,NaN,2017-01,2019-12,2017-01,2019-12,2020-02,4,4,NaN


In [ ]:
df_base_5.tail()

,periodo,customer_id,product_id,plan_precios_cuidados,tn,dif_tn,dif_por,anio,mes,secuencia,trimestre,date,date_mes,cat1,cat1_cat2,cat1_cat2_cat3,cat1_cat2_cat3_brand,cat1_cat2_cat3_brand_size,stock_final,primera_venta,ultima_venta,alta_cliente,ultima_compra,vive,meses_vida_producto,meses_vida_cliente,meses_vida_prod
2945813,201912,10105,20853,False,0.022,0.0,0.0,2019,12,36,4,2019-12-01,2019-12,PC,PC_CABELLO,PC_CABELLO_Shampoo Bebe,PC_CABELLO_Shampoo Bebe_NIVEA,PC_CABELLO_Shampoo Bebe_NIVEA_200.0,1.824,2018-08,2019-12,2017-01,2019-12,2020-02,17,39,NaN
2945814,201912,10092,20853,False,0.007,0.0,0.0,2019,12,36,4,2019-12-01,2019-12,PC,PC_CABELLO,PC_CABELLO_Shampoo Bebe,PC_CABELLO_Shampoo Bebe_NIVEA,PC_CABELLO_Shampoo Bebe_NIVEA_200.0,1.824,2018-08,2019-12,2017-01,2019-12,2020-02,17,39,NaN
2945815,201912,10006,20853,False,0.029,0.0,0.0,2019,12,36,4,2019-12-01,2019-12,PC,PC_CABELLO,PC_CABELLO_Shampoo Bebe,PC_CABELLO_Shampoo Bebe_NIVEA,PC_CABELLO_Shampoo Bebe_NIVEA_200.0,1.824,2018-08,2019-12,2017-01,2019-12,2020-02,17,39,NaN
2945816,201912,10018,20853,False,0.016,0.0,0.0,2019,12,36,4,2019-12-01,2019-12,PC,PC_CABELLO,PC_CABELLO_Shampoo Bebe,PC_CABELLO_Shampoo Bebe_NIVEA,PC_CABELLO_Shampoo Bebe_NIVEA_200.0,1.824,2018-08,2019-12,2017-01,2019-12,2020-02,17,39,NaN
2945817,201912,10020,20853,False,0.016,0.0,0.0,2019,12,36,4,2019-12-01,2019-12,PC,PC_CABELLO,PC_CABELLO_Shampoo Bebe,PC_CABELLO_Shampoo Bebe_NIVEA,PC_CABELLO_Shampoo Bebe_NIVEA_200.0,1.824,2018-08,2019-12,2017-01,2019-12,2020-02,17,39,NaN


In [ ]:
df_base_5['meses_vida_prod'].unique()

array([nan, 36., 17., 12.,  6.,  0., 20., 25., 16., 15., 27., 28., 32.,
        8., 13.,  5., 14., 26., 11., 23., 31., 10., 30., 29.,  7.,  9.,
       34.,  2.,  1., 22., 19., 24., 33., 21., 18.,  4., 35.,  3.])

In [ ]:
df_base_5.shape

(2945818, 27)

In [ ]:
# ahora quiero agregar una columna que sea simplemente la cantidad de meses desde que este cliente compró por última vez este producto
# pero si hago esto tengo que explotar la base con todos los clientes y todos los productos y los NA
# en principio lo podría hacer pero lo dejo para verlo cuando tenga las series cliente producto. hasta ahora sigo por producto.
df_base_5.isnull().sum()

,0
periodo,0
customer_id,0
product_id,0
plan_precios_cuidados,0
tn,0
dif_tn,0
dif_por,0
anio,0
mes,0
secuencia,0


la df_base_5 me quedó entonces con los datos que considero interesantes para tener en una agregación a nivel producto

### Totales por parciales y categorias

In [ ]:
# no tengo nada vacío, OJO hay ceros que tengo que pasar a NA si quiero procesar OJO
df_base_5.to_csv(DATOS_DIR +'df_base_5.csv', index=False)

In [ ]:
# Vamos entonces a hacer tablas resumen de forma tal de
#### por un lado tener las tablas a disposición
#### y luego poder unirlas para tener un dataset
# Lista para almacenar nuestras tablas de resumen antes de unirlas
lista_resumenes_gral = []

# 2. CALCULAR Y ESTANDARIZAR CADA RESUMEN

# A. Total de Ventas General
resumen_general = df_base_5.groupby('date_mes')['tn'].sum().reset_index()
resumen_general['dimension'] = 'Total General'
resumen_general['item'] = 'Total General'  # El 'item' es la propia dimensión
resumen_general = resumen_general.rename(columns={'tn': 'total_tn'})
resumen_general.to_csv(DATOS_DIR +'resumen_general.csv', index=False)
lista_resumenes_gral.append(resumen_general)
#print(resumen_general.head())

# B. Total de Ventas Clientes
resumen_cliente = df_base_5.groupby(['date_mes', 'customer_id'])['tn'].sum().reset_index()
resumen_cliente['dimension'] = 'Por Cliente'
resumen_cliente['item'] = 'Total Cliente'  # El 'item' es la propia dimensión
resumen_cliente = resumen_cliente.rename(columns={'tn': 'total_tn'})
resumen_cliente.to_csv(DATOS_DIR +'resumen_cliente.csv', index=False)
lista_resumenes_gral.append(resumen_cliente)
#print(resumen_cliente.head())

# C. Total de Ventas producto
resumen_producto = df_base_5.groupby(['product_id', 'date_mes'])['tn'].sum().reset_index()
resumen_producto['dimension'] = 'Total Producto'
resumen_producto['item'] = 'Total Producto'  # El 'item' es la propia dimensión
resumen_producto = resumen_producto.rename(columns={'tn': 'total_tn'})
resumen_producto.to_csv(DATOS_DIR +'resumen_producto.csv', index=False)
lista_resumenes_gral.append(resumen_producto)
#print(resumen_producto.head())

# D. Total de Ventas cat1
resumen_cat1 = df_base_5.groupby(['cat1', 'date_mes'])['tn'].sum().reset_index()
resumen_cat1['dimension'] = 'Total Cat1'
resumen_cat1['item'] = 'Total Cat1'  # El 'item' es la propia dimensión
resumen_cat1 = resumen_cat1.rename(columns={'tn': 'total_tn'})
resumen_cat1.to_csv(DATOS_DIR +'resumen_cat1.csv', index=False)
lista_resumenes_gral.append(resumen_cat1)
#print(resumen_cat1.head())

# E. Total de Ventas cat1_cat2
resumen_cat1_cat2 = df_base_5.groupby(['cat1_cat2', 'date_mes'])['tn'].sum().reset_index()
resumen_cat1_cat2['dimension'] = 'Total Cat1_Cat2'
resumen_cat1_cat2['item'] = 'Total Cat1_Cat2'  # El 'item' es la propia
resumen_cat1_cat2 = resumen_cat1_cat2.rename(columns={'tn': 'total_tn'})
resumen_cat1_cat2.to_csv(DATOS_DIR +'resumen_cat1_cat2.csv', index=False)
lista_resumenes_gral.append(resumen_cat1_cat2)
#print(resumen_cat1_cat2.head())

# F. Total de Ventas cat1_cat2_cat3
resumen_cat1_cat2_cat3 = df_base_5.groupby(['cat1_cat2_cat3', 'date_mes'])['tn'].sum().reset_index()
resumen_cat1_cat2_cat3['dimension'] = 'Total Cat1_Cat2_Cat3'
resumen_cat1_cat2_cat3['item'] = 'Total Cat1_Cat2_Cat3'  # El 'item' es la propia dimensión
resumen_cat1_cat2_cat3 = resumen_cat1_cat2_cat3.rename(columns={'tn': 'total_tn'})
resumen_cat1_cat2_cat3.to_csv(DATOS_DIR +'resumen_cat1_cat2_cat3.csv', index=False)
lista_resumenes_gral.append(resumen_cat1_cat2_cat3)
#print(resumen_cat1_cat2_cat3.head())

# G. Total de Ventas cat1_cat2_cat3_brand
resumen_cat1_cat2_cat3_brand = df_base_5.groupby(['cat1_cat2_cat3_brand', 'date_mes'])['tn'].sum().reset_index()
resumen_cat1_cat2_cat3_brand['dimension'] = 'Total Cat1_Cat2_Cat3_Brand'
resumen_cat1_cat2_cat3_brand['item'] = 'Total Cat1_Cat2_Cat3_Brand'  # El 'item' es la propia dimensión
resumen_cat1_cat2_cat3_brand = resumen_cat1_cat2_cat3_brand.rename(columns={'tn': 'total_tn'})
resumen_cat1_cat2_cat3_brand.to_csv(DATOS_DIR +'resumen_cat1_cat2_cat3_brand.csv', index=False)
lista_resumenes_gral.append(resumen_cat1_cat2_cat3_brand)
#print(resumen_cat1_cat2_cat3_brand.head())

# H. Total de Ventas cat1_cat2_cat3_brand_size
resumen_cat1_cat2_cat3_brand_size = df_base_5.groupby(['cat1_cat2_cat3_brand_size', 'date_mes'])['tn'].sum().reset_index()
resumen_cat1_cat2_cat3_brand_size['dimension'] = 'Total Cat1_Cat2_Cat3_Brand_Size'
resumen_cat1_cat2_cat3_brand_size['item'] = 'Total Cat1_Cat2_Cat3_Brand_Size'  # El 'item' es la propia
resumen_cat1_cat2_cat3_brand_size = resumen_cat1_cat2_cat3_brand_size.rename(columns={'tn': 'total_tn'})
resumen_cat1_cat2_cat3_brand_size.to_csv(DATOS_DIR +'resumen_cat1_cat2_cat3_brand_size.csv', index=False)
lista_resumenes_gral.append(resumen_cat1_cat2_cat3_brand_size)
#print(resumen_cat1_cat2_cat3_brand_size.head())


In [ ]:
# Todas las tablas en la lista tienen una estructura compatible.
df_resumen_tot = pd.concat(lista_resumenes_gral, ignore_index=True)
df_resumen_tot.head()


,date_mes,total_tn,dimension,item,customer_id,product_id,cat1,cat1_cat2,cat1_cat2_cat3,cat1_cat2_cat3_brand,cat1_cat2_cat3_brand_size
0,2017-01,34057.318,Total General,Total General,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2017-02,34568.653,Total General,Total General,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2017-03,46040.596,Total General,Total General,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2017-04,39625.524,Total General,Total General,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2017-05,45579.633,Total General,Total General,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
for col in df_resumen_tot.columns:
    print(f"Columna: {col}")
    print(df_resumen_tot[col].unique())
    print("-" * 40)

Columna: date_mes
<PeriodArray>
['2017-01', '2017-02', '2017-03', '2017-04', '2017-05', '2017-06', '2017-07',
 '2017-08', '2017-09', '2017-10', '2017-11', '2017-12', '2018-01', '2018-02',
 '2018-03', '2018-04', '2018-05', '2018-06', '2018-07', '2018-08', '2018-09',
 '2018-10', '2018-11', '2018-12', '2019-01', '2019-02', '2019-03', '2019-04',
 '2019-05', '2019-06', '2019-07', '2019-08', '2019-09', '2019-10', '2019-11',
 '2019-12']
Length: 36, dtype: period[M]
----------------------------------------
Columna: total_tn
[3.40573178e+04 3.45686531e+04 4.60405964e+04 ... 5.18350000e-01
 1.21600000e-01 9.68400000e-02]
----------------------------------------
Columna: dimension
['Total General' 'Por Cliente' 'Total Producto' 'Total Cat1'
 'Total Cat1_Cat2' 'Total Cat1_Cat2_Cat3' 'Total Cat1_Cat2_Cat3_Brand'
 'Total Cat1_Cat2_Cat3_Brand_Size']
----------------------------------------
Columna: item
['Total General' 'Total Cliente' 'Total Producto' 'Total Cat1'
 'Total Cat1_Cat2' 'Total Cat1_Cat2

In [ ]:
df_resumen_tot.shape

(67089, 11)

In [ ]:
print("--- Tabla Resumen Final (versión corregida) ---")
print(df_resumen_tot.head())
df_resumen_tot.to_csv(DATOS_DIR +'df_resumen_tot.csv', index=False)

--- Tabla Resumen Final (versión corregida) ---
  date_mes   total_tn      dimension           item  customer_id  product_id  \
0  2017-01  34057.318  Total General  Total General          NaN         NaN   
1  2017-02  34568.653  Total General  Total General          NaN         NaN   
2  2017-03  46040.596  Total General  Total General          NaN         NaN   
3  2017-04  39625.524  Total General  Total General          NaN         NaN   
4  2017-05  45579.633  Total General  Total General          NaN         NaN   

  cat1 cat1_cat2 cat1_cat2_cat3 cat1_cat2_cat3_brand cat1_cat2_cat3_brand_size  
0  NaN       NaN            NaN                  NaN                       NaN  
1  NaN       NaN            NaN                  NaN                       NaN  
2  NaN       NaN            NaN                  NaN                       NaN  
3  NaN       NaN            NaN                  NaN                       NaN  
4  NaN       NaN            NaN                  NaN              

### Agregando rankings

In [ ]:
df_base_6 = df_base_5.copy()

In [ ]:
# 1. Ranking por producto por mes y share por mes (participación sobre el total del mes)
df_base_6['ventas_producto_mes'] = df_base_6.groupby(['date_mes', 'product_id'])['tn'].transform('sum')
df_base_6['total_mes'] = df_base_6.groupby('date_mes')['tn'].transform('sum')

# Ranking (1 = el que más vendió ese mes)
df_base_6['rank_producto_mes'] = df_base_6.groupby('date_mes')['ventas_producto_mes'].rank(method='first', ascending=False).astype(int)

# Share
df_base_6['share_producto_mes'] =100* df_base_6['ventas_producto_mes'] / df_base_6['total_mes']
df_base_6.head()

,periodo,customer_id,product_id,plan_precios_cuidados,tn,dif_tn,dif_por,anio,mes,secuencia,trimestre,date,date_mes,cat1,cat1_cat2,cat1_cat2_cat3,cat1_cat2_cat3_brand,cat1_cat2_cat3_brand_size,stock_final,primera_venta,ultima_venta,alta_cliente,ultima_compra,vive,meses_vida_producto,meses_vida_cliente,meses_vida_prod,ventas_producto_mes,total_mes,rank_producto_mes,share_producto_mes
0,201701,10234,20524,False,0.053,0.0,0.0,2017,1,1,0,2017-01-01,2017-01,HC,HC_VAJILLA,HC_VAJILLA_Cristalino,HC_VAJILLA_Cristalino_Importado,HC_VAJILLA_Cristalino_Importado_500.0,NaN,2017-01,2019-12,2017-01,2019-12,2020-02,4,4,NaN,6.481,34057.318,58667,0.019
1,201701,10032,20524,False,0.136,0.0,0.0,2017,1,1,0,2017-01-01,2017-01,HC,HC_VAJILLA,HC_VAJILLA_Cristalino,HC_VAJILLA_Cristalino_Importado,HC_VAJILLA_Cristalino_Importado_500.0,NaN,2017-01,2019-12,2017-01,2019-12,2020-02,4,4,NaN,6.481,34057.318,58668,0.019
2,201701,10217,20524,False,0.030,0.0,0.0,2017,1,1,0,2017-01-01,2017-01,HC,HC_VAJILLA,HC_VAJILLA_Cristalino,HC_VAJILLA_Cristalino_Importado,HC_VAJILLA_Cristalino_Importado_500.0,NaN,2017-01,2019-12,2017-01,2019-12,2020-02,4,4,NaN,6.481,34057.318,58669,0.019
3,201701,10125,20524,False,0.023,0.0,0.0,2017,1,1,0,2017-01-01,2017-01,HC,HC_VAJILLA,HC_VAJILLA_Cristalino,HC_VAJILLA_Cristalino_Importado,HC_VAJILLA_Cristalino_Importado_500.0,NaN,2017-01,2019-12,2017-01,2019-11,2020-02,4,4,NaN,6.481,34057.318,58670,0.019
4,201701,10012,20524,False,1.545,0.0,0.0,2017,1,1,0,2017-01-01,2017-01,HC,HC_VAJILLA,HC_VAJILLA_Cristalino,HC_VAJILLA_Cristalino_Importado,HC_VAJILLA_Cristalino_Importado_500.0,NaN,2017-01,2019-12,2017-01,2019-12,2020-02,4,4,NaN,6.481,34057.318,58671,0.019


In [ ]:
# 1. Ranking por cliente por mes y share por mes (participación sobre el total del mes)
df_base_6['ventas_cliente_mes'] = df_base_6.groupby(['date_mes', 'customer_id'])['tn'].transform('sum')

# Ranking (1 = el que más vendió ese mes)
df_base_6['rank_cliente_mes'] = df_base_6.groupby('date_mes')['ventas_cliente_mes'].rank(method='first', ascending=False).astype(int)

# Share
df_base_6['share_cliente_mes'] =100* df_base_6['ventas_cliente_mes'] / df_base_6['total_mes']
df_base_6.head()

,periodo,customer_id,product_id,plan_precios_cuidados,tn,dif_tn,dif_por,anio,mes,secuencia,trimestre,date,date_mes,cat1,cat1_cat2,cat1_cat2_cat3,cat1_cat2_cat3_brand,cat1_cat2_cat3_brand_size,stock_final,primera_venta,ultima_venta,alta_cliente,ultima_compra,vive,meses_vida_producto,meses_vida_cliente,meses_vida_prod,ventas_producto_mes,total_mes,rank_producto_mes,share_producto_mes,ventas_cliente_mes,rank_cliente_mes,share_cliente_mes
0,201701,10234,20524,False,0.053,0.0,0.0,2017,1,1,0,2017-01-01,2017-01,HC,HC_VAJILLA,HC_VAJILLA_Cristalino,HC_VAJILLA_Cristalino_Importado,HC_VAJILLA_Cristalino_Importado_500.0,NaN,2017-01,2019-12,2017-01,2019-12,2020-02,4,4,NaN,6.481,34057.318,58667,0.019,21.640,46045,0.064
1,201701,10032,20524,False,0.136,0.0,0.0,2017,1,1,0,2017-01-01,2017-01,HC,HC_VAJILLA,HC_VAJILLA_Cristalino,HC_VAJILLA_Cristalino_Importado,HC_VAJILLA_Cristalino_Importado_500.0,NaN,2017-01,2019-12,2017-01,2019-12,2020-02,4,4,NaN,6.481,34057.318,58668,0.019,299.410,10223,0.879
2,201701,10217,20524,False,0.030,0.0,0.0,2017,1,1,0,2017-01-01,2017-01,HC,HC_VAJILLA,HC_VAJILLA_Cristalino,HC_VAJILLA_Cristalino_Importado,HC_VAJILLA_Cristalino_Importado_500.0,NaN,2017-01,2019-12,2017-01,2019-12,2020-02,4,4,NaN,6.481,34057.318,58669,0.019,18.067,48053,0.053
3,201701,10125,20524,False,0.023,0.0,0.0,2017,1,1,0,2017-01-01,2017-01,HC,HC_VAJILLA,HC_VAJILLA_Cristalino,HC_VAJILLA_Cristalino_Importado,HC_VAJILLA_Cristalino_Importado_500.0,NaN,2017-01,2019-12,2017-01,2019-11,2020-02,4,4,NaN,6.481,34057.318,58670,0.019,15.985,49651,0.047
4,201701,10012,20524,False,1.545,0.0,0.0,2017,1,1,0,2017-01-01,2017-01,HC,HC_VAJILLA,HC_VAJILLA_Cristalino,HC_VAJILLA_Cristalino_Importado,HC_VAJILLA_Cristalino_Importado_500.0,NaN,2017-01,2019-12,2017-01,2019-12,2020-02,4,4,NaN,6.481,34057.318,58671,0.019,907.775,4779,2.665


In [ ]:
df_base_6.shape

(2945818, 34)

In [ ]:
df_base_6.to_csv(DATOS_DIR +'df_base_6.csv', index=False)

In [ ]:
# Estoy basado en producto, cliente producto dije que va para otro lado. Pero igual voy a hacer todos los rankings y shares
# este no va porque apila mal, me junta todas las categorías lo dejo hasta el punto anterior.
# hacer esto requiere la podrida  tabla larga

# contra cat1
df_base_6['ventas_producto_cat1_mes'] = df_base_6.groupby(['date_mes', 'product_id', 'cat1'])['tn'].transform('sum')
df_base_6['total_cat1_mes'] = df_base_6.groupby(['date_mes','cat1'])['tn'].transform('sum')

# Ranking (1 = el que más vendió ese mes)
df_base_6['rank_producto_cat1_mes'] = df_base_6.groupby('date_mes')['ventas_producto_cat1_mes'].rank(method='first', ascending=False).astype(int)

# Share
df_base_6['share_producto_cat1_mes'] =100* df_base_6['ventas_producto_cat1_mes'] / df_base_6['total_cat1_mes']
df_base_6.head()

,periodo,customer_id,product_id,plan_precios_cuidados,tn,dif_tn,dif_por,anio,mes,secuencia,trimestre,date,date_mes,cat1,cat1_cat2,cat1_cat2_cat3,cat1_cat2_cat3_brand,cat1_cat2_cat3_brand_size,stock_final,primera_venta,ultima_venta,alta_cliente,ultima_compra,vive,meses_vida_producto,meses_vida_cliente,meses_vida_prod,ventas_producto_mes,total_mes,rank_producto_mes,share_producto_mes,ventas_cliente_mes,rank_cliente_mes,share_cliente_mes,ventas_producto_cat1_mes,total_cat1_mes,rank_producto_cat1_mes,share_producto_cat1_mes
0,201701,10234,20524,False,0.053,0.0,0.0,2017,1,1,0,2017-01-01,2017-01,HC,HC_VAJILLA,HC_VAJILLA_Cristalino,HC_VAJILLA_Cristalino_Importado,HC_VAJILLA_Cristalino_Importado_500.0,NaN,2017-01,2019-12,2017-01,2019-12,2020-02,4,4,NaN,6.481,34057.318,58667,0.019,21.640,46045,0.064,6.481,20304.287,58667,0.032
1,201701,10032,20524,False,0.136,0.0,0.0,2017,1,1,0,2017-01-01,2017-01,HC,HC_VAJILLA,HC_VAJILLA_Cristalino,HC_VAJILLA_Cristalino_Importado,HC_VAJILLA_Cristalino_Importado_500.0,NaN,2017-01,2019-12,2017-01,2019-12,2020-02,4,4,NaN,6.481,34057.318,58668,0.019,299.410,10223,0.879,6.481,20304.287,58668,0.032
2,201701,10217,20524,False,0.030,0.0,0.0,2017,1,1,0,2017-01-01,2017-01,HC,HC_VAJILLA,HC_VAJILLA_Cristalino,HC_VAJILLA_Cristalino_Importado,HC_VAJILLA_Cristalino_Importado_500.0,NaN,2017-01,2019-12,2017-01,2019-12,2020-02,4,4,NaN,6.481,34057.318,58669,0.019,18.067,48053,0.053,6.481,20304.287,58669,0.032
3,201701,10125,20524,False,0.023,0.0,0.0,2017,1,1,0,2017-01-01,2017-01,HC,HC_VAJILLA,HC_VAJILLA_Cristalino,HC_VAJILLA_Cristalino_Importado,HC_VAJILLA_Cristalino_Importado_500.0,NaN,2017-01,2019-12,2017-01,2019-11,2020-02,4,4,NaN,6.481,34057.318,58670,0.019,15.985,49651,0.047,6.481,20304.287,58670,0.032
4,201701,10012,20524,False,1.545,0.0,0.0,2017,1,1,0,2017-01-01,2017-01,HC,HC_VAJILLA,HC_VAJILLA_Cristalino,HC_VAJILLA_Cristalino_Importado,HC_VAJILLA_Cristalino_Importado_500.0,NaN,2017-01,2019-12,2017-01,2019-12,2020-02,4,4,NaN,6.481,34057.318,58671,0.019,907.775,4779,2.665,6.481,20304.287,58671,0.032


In [ ]:
# contra cat1_cat2
# contra cat1_cat2_cat3
# contra cat1_cat2_cat3_brand
# contra cat1_cat2_cat3_brand_size

### cliente producto queda para otro colab

In [ ]:
# alargo base y guardo


# 4.- Datos Argentina

In [ ]:

# me traigo la última df con los datos, la pelo de columna, le elimino las filas duplicadas, y empiezo a agregar columnas
df_base_6.head()

,periodo,customer_id,product_id,plan_precios_cuidados,tn,dif_tn,dif_por,anio,mes,secuencia,trimestre,date,date_mes,cat1,cat1_cat2,cat1_cat2_cat3,cat1_cat2_cat3_brand,cat1_cat2_cat3_brand_size,stock_final,primera_venta,ultima_venta,alta_cliente,ultima_compra,vive,meses_vida_producto,meses_vida_cliente,meses_vida_prod,ventas_producto_mes,total_mes,rank_producto_mes,share_producto_mes,ventas_cliente_mes,rank_cliente_mes,share_cliente_mes
0,201701,10234,20524,False,0.053,0.0,0.0,2017,1,1,0,2017-01-01,2017-01,HC,HC_VAJILLA,HC_VAJILLA_Cristalino,HC_VAJILLA_Cristalino_Importado,HC_VAJILLA_Cristalino_Importado_500.0,NaN,2017-01,2019-12,2017-01,2019-12,2020-02,4,4,NaN,6.481,34057.318,58667,0.019,21.640,46045,0.064
1,201701,10032,20524,False,0.136,0.0,0.0,2017,1,1,0,2017-01-01,2017-01,HC,HC_VAJILLA,HC_VAJILLA_Cristalino,HC_VAJILLA_Cristalino_Importado,HC_VAJILLA_Cristalino_Importado_500.0,NaN,2017-01,2019-12,2017-01,2019-12,2020-02,4,4,NaN,6.481,34057.318,58668,0.019,299.410,10223,0.879
2,201701,10217,20524,False,0.030,0.0,0.0,2017,1,1,0,2017-01-01,2017-01,HC,HC_VAJILLA,HC_VAJILLA_Cristalino,HC_VAJILLA_Cristalino_Importado,HC_VAJILLA_Cristalino_Importado_500.0,NaN,2017-01,2019-12,2017-01,2019-12,2020-02,4,4,NaN,6.481,34057.318,58669,0.019,18.067,48053,0.053
3,201701,10125,20524,False,0.023,0.0,0.0,2017,1,1,0,2017-01-01,2017-01,HC,HC_VAJILLA,HC_VAJILLA_Cristalino,HC_VAJILLA_Cristalino_Importado,HC_VAJILLA_Cristalino_Importado_500.0,NaN,2017-01,2019-12,2017-01,2019-11,2020-02,4,4,NaN,6.481,34057.318,58670,0.019,15.985,49651,0.047
4,201701,10012,20524,False,1.545,0.0,0.0,2017,1,1,0,2017-01-01,2017-01,HC,HC_VAJILLA,HC_VAJILLA_Cristalino,HC_VAJILLA_Cristalino_Importado,HC_VAJILLA_Cristalino_Importado_500.0,NaN,2017-01,2019-12,2017-01,2019-12,2020-02,4,4,NaN,6.481,34057.318,58671,0.019,907.775,4779,2.665


In [ ]:
columnas_a_conservar = ['periodo','date','date_mes']
df_base_20 = df_base_6[columnas_a_conservar]
df_base_20 =df_base_20.drop_duplicates()
df_base_20 = df_base_20.reset_index(drop=True)
df_base_20.head()

,periodo,date,date_mes
0,201701,2017-01-01,2017-01
1,201702,2017-02-01,2017-02
2,201703,2017-03-01,2017-03
3,201704,2017-04-01,2017-04
4,201705,2017-05-01,2017-05


In [ ]:
df_base_20.tail()

,periodo,date,date_mes
31,201908,2019-08-01,2019-08
32,201909,2019-09-01,2019-09
33,201910,2019-10-01,2019-10
34,201911,2019-11-01,2019-11
35,201912,2019-12-01,2019-12


In [ ]:
nuevos_periodos = [202001, 202002]
nuevas_filas = []

for periodo in nuevos_periodos:
    anio = int(str(periodo)[:4])
    mes = int(str(periodo)[4:])
    date = f"{anio}-{mes:02d}-01"
    date_mes = f"{anio}-{mes:02d}"
    nuevas_filas.append({'periodo': periodo, 'date': date, 'date_mes': date_mes})

df_nuevas = pd.DataFrame(nuevas_filas)

In [ ]:
df_base_20= pd.concat([df_base_20, df_nuevas], ignore_index=True)
df_base_20.tail()

,periodo,date,date_mes
33,201910,2019-10-01 00:00:00,2019-10
34,201911,2019-11-01 00:00:00,2019-11
35,201912,2019-12-01 00:00:00,2019-12
36,202001,2020-01-01,2020-01
37,202002,2020-02-01,2020-02


In [ ]:
# agrego días laborables
# Instancia de feriados argentinos para el año necesario
arg_feriados = holidays.country_holidays('AR')

def dias_laborables_argentina(mes):
    year = mes.year
    month = mes.month
    # Primer y último día del mes
    inicio = pd.Timestamp(year, month, 1)
    fin = (inicio + pd.offsets.MonthEnd(0))
    # Genera rango de fechas laborables (lunes-viernes)
    dias = pd.date_range(inicio, fin, freq='B')
    # Excluye feriados
    feriados_en_mes = [d for d in dias if d in arg_feriados]
    return len(dias) - len(feriados_en_mes)

# Convert 'date_mes' to Period type before applying the function
df_base_20['date_mes'] = pd.PeriodIndex(df_base_20['date_mes'], freq='M')

# Aplica la función
df_base_20['dias_laborables_arg'] = df_base_20['date_mes'].apply(dias_laborables_argentina)
print(df_base_20.head())

   periodo                 date date_mes  dias_laborables_arg
0   201701  2017-01-01 00:00:00  2017-01                   22
1   201702  2017-02-01 00:00:00  2017-02                   18
2   201703  2017-03-01 00:00:00  2017-03                   22
3   201704  2017-04-01 00:00:00  2017-04                   18
4   201705  2017-05-01 00:00:00  2017-05                   21


In [ ]:
# cisnes negros :elecciones eventos
df_base_20['cisne_negro'] = 0  # crea la columna con ceros
df_base_20.loc[df_base_20['date_mes'] == '2018-08', 'cisne_negro'] = 1  # pone 1 solo en esa condición
print(df_base_20.head())

   periodo                 date date_mes  dias_laborables_arg  cisne_negro
0   201701  2017-01-01 00:00:00  2017-01                   22            0
1   201702  2017-02-01 00:00:00  2017-02                   18            0
2   201703  2017-03-01 00:00:00  2017-03                   22            0
3   201704  2017-04-01 00:00:00  2017-04                   18            0
4   201705  2017-05-01 00:00:00  2017-05                   21            0


In [ ]:
# consumo general
# cargo los stocks de los productos
ESt_arg = pd.read_csv(DATOS_DIR +'estad_ARG_1.csv')

In [ ]:
ESt_arg.tail()

,periodo,VtaGral,VtaGralCte,Bebidas,Almacén,Panadería,Lácteos,Carnes,Verdulería y frutería,Artículos de limpieza y perfumería,"Indumentaria, calzado y textiles para el hogar",Electrónicos y artículos para el hogar,Otros,USD_of,USD_blue,Infl,Expect
31,201908,55044.48,23580.07,9.921e+05,4.073e+06,19093.96,621666.85,128557.64,18930.57,2.611e+06,2695.75,162132.54,516909.38,42.90,60.00,2.2,41.86
32,201909,59672.10,24329.91,1.001e+06,4.609e+06,21437.17,648000.18,145708.62,21414.39,2.672e+06,2143.30,131656.15,507576.71,57.00,58.25,4.0,42.09
33,201910,57885.31,22144.65,9.910e+05,4.356e+06,21457.63,664708.72,146363.94,20634.75,2.824e+06,2505.22,154919.68,515664.66,55.50,66.00,5.9,43.77
34,201911,66325.58,24652.71,1.263e+06,4.782e+06,23395.00,739381.36,158337.36,21758.64,3.203e+06,2948.21,230350.11,581451.68,58.22,66.25,3.3,41.35
35,201912,67943.71,24270.70,1.519e+06,4.849e+06,26998.04,670188.53,156887.10,22055.64,3.266e+06,2703.85,260444.52,609248.93,NaN,NaN,NaN,NaN


In [ ]:
ESt_arg['periodo']= ESt_arg['periodo'].astype(int)

In [ ]:
# consumo por categoría
# Unir agregando columnas de df2 a df1 por la columna 'periodo'
df_base_20 = pd.merge(df_base_20, ESt_arg, on='periodo', how='left')
df_base_20.head()

,periodo,date,date_mes,dias_laborables_arg,cisne_negro,VtaGral,VtaGralCte,Bebidas,Almacén,Panadería,Lácteos,Carnes,Verdulería y frutería,Artículos de limpieza y perfumería,"Indumentaria, calzado y textiles para el hogar",Electrónicos y artículos para el hogar,Otros,USD_of,USD_blue,Infl,Expect
0,201701,2017-01-01 00:00:00,2017-01,22,0,27858.87,27477.31,573064.06,1.624e+06,11576.02,253462.37,73883.84,20142.56,1.219e+06,1146.38,48728.12,269322.15,15.7,16.47,1.6,44.47
1,201702,2017-02-01 00:00:00,2017-02,18,0,27858.87,27477.31,573064.06,1.624e+06,11576.02,253462.37,73883.84,20142.56,1.219e+06,1146.38,48728.12,269322.15,15.7,16.24,1.6,40.68
2,201703,2017-03-01 00:00:00,2017-03,22,0,25994.20,25223.43,540122.06,1.559e+06,11395.77,252720.27,75545.85,20456.38,1.089e+06,1018.56,34128.35,257899.92,15.3,15.88,2.1,40.95
3,201704,2017-04-01 00:00:00,2017-04,18,0,27780.01,26334.26,586230.17,2.006e+06,12889.83,307951.54,89305.98,22607.84,1.296e+06,927.07,36321.56,301861.90,15.2,15.59,2.4,46.19
4,201705,2017-05-01 00:00:00,2017-05,21,0,29134.27,27045.88,572661.71,1.967e+06,13642.25,298638.09,84722.68,24054.43,1.219e+06,1098.13,43054.66,298741.07,15.2,15.99,2.7,45.76


# 5.- Guarda de resultados


In [ ]:
df_base_20.to_csv(DATOS_DIR +'df_base_20.csv', index=False)